In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv("/content/gdrive/My Drive/CS 5350/Homework 2/concrete/train.csv")
train_x = train.iloc[:, :-1].to_numpy()
train_y = train.iloc[:, -1].to_numpy()
test = pd.read_csv("/content/gdrive/My Drive/CS 5350/Homework 2/concrete/train.csv")
test_x = test.iloc[:, :-1].to_numpy()
test_y = test.iloc[:, -1].to_numpy()

We're doing the bias trick here.

In [3]:
new_train = []
for row in train_x:
  new_train.append(np.insert(row, 0, np.array([1])))
new_train = np.array(new_train)
print(new_train)

[[ 1.00000e+00  5.49165e-01  6.69171e-02 -5.17804e-01  6.38098e-01
   1.64716e-01  2.27616e-01 -9.45600e-01]
 [ 1.00000e+00  9.41555e-01 -1.29263e+00  1.06803e+00  1.95089e+00
  -8.58329e-02 -1.18096e+00 -3.85584e-01]
 [ 1.00000e+00 -9.66886e-01  1.04752e+00  3.76289e-01  1.83154e+00
   8.80569e-01 -9.88829e-01 -8.98007e-01]
 [ 1.00000e+00  6.38344e-01  2.33121e-01 -3.64867e-01  4.88917e-01
   1.64716e-01 -1.11258e-02 -9.61465e-01]
 [ 1.00000e+00  1.05876e+00  7.64973e-01 -1.75306e+00  3.89463e-01
   8.80569e-01 -4.31766e-01  2.91829e-01]
 [ 1.00000e+00 -8.90447e-01  1.13062e+00  4.58639e-01  2.90009e-01
   8.80569e-01 -6.25032e-01 -4.69666e-01]
 [ 1.00000e+00 -1.82107e-01  5.80486e-01  1.05862e+00  7.61836e-02
  -4.07967e-01  2.42825e-04 -1.43581e+00]
 [ 1.00000e+00 -1.14270e+00 -5.51360e-01  1.01039e+00 -1.28633e+00
  -1.08803e+00  1.86015e+00 -5.64853e-01]
 [ 1.00000e+00 -8.52227e-01  1.18048e+00  4.93932e-01 -8.53710e-01
   1.23850e+00 -4.65871e-01  1.01456e-01]
 [ 1.00000e+00  1.0

In [4]:
# Cost function for an inividual point
def cost_function(data, model, actual_y):
  return (data.T @ model - actual_y) ** 2

# Total cost for some data
def total_cost_function(data, model, y):
  total_cost = 0
  for i in range(len(train_x)):
    total_cost += cost_function(new_train[i], model, train_y[i])
  return total_cost

The gradient at a single point:<br>
$$<2 * M(\alpha) \cdot x * x_{1}, ...,  2 * M(\alpha) \cdot x * x_{n}>$$ Where the angle brakets define a vector.<br>
$$\sum_{n=1}^{n}<2 * M(\alpha) \cdot x_n * x_{n, 1}, ...,  2 * M(\alpha) \cdot x_n * x_{n, j}>$$


In [34]:
# Gradient function for a single point
def gradient(data, model, actual_y):
  return np.array([2 * (data.T @ model - actual_y) * data[0], 2 * (data.T @ model - actual_y) * data[1], 2 * (data.T @ model - actual_y) * data[2], 2 * (data.T @ model - actual_y) * data[3]])

# Function describing batch gradient
def batch_gradient_2(data, model, y):
  sum_grad = 0
  for i in range(len(new_train)):
    sum_grad += gradient(data[i], model, train_y[i])
  return sum_grad

def batch_gradient(x, model, y):
  resid = 0
  for i in range(len(x)):
    resid += (new_train[i].T @ model - train_y[i]) * x[i]
  return 2 * resid

This is another way to describe the batch gradient.<br>
$$2*\sum_{n=1}^{n} (M(α) \cdot x_n - y_n)*(1, x_{j, 1}, ..., x_{j, n})$$

Time for batch gradient descent:

In [26]:
num_iter = 5_000
gamma = 0.008

In [27]:
from numpy import linalg as LA

queue = []
params = np.zeros(new_train.shape[1])
tao = 0.0000001
for i in range(num_iter):
    grad = batch_gradient(new_train, params, train_y)
    SSE = total_cost_function(new_train, params, train_y)
    grad_norm = LA.norm(grad)
    params = params - gamma * grad
    print("SSE:", SSE, "grad-norm:", grad_norm, "parameters", params)

    # This handles the stopping condition.
    queue.append(grad_norm)
    if len(queue) == 10 and np.mean(queue) < tao:
      break
    elif len(queue) == 10:
      queue.pop(0)

Streaming output truncated to the last 5000 lines.
SSE: 29.758417404911906 grad-norm: 0.008049957984185964 parameters [-0.02224697  0.98414454  0.88343511  0.94155538  1.37089426  0.12636281
  1.67478163  1.08616031]
SSE: 29.75841688694831 grad-norm: 0.008035951503388764 parameters [-0.02224654  0.98416917  0.88346059  0.94158278  1.37091398  0.12636678
  1.67481463  1.08618547]
SSE: 29.75841637078563 grad-norm: 0.008021969393090203 parameters [-0.02224612  0.98419376  0.88348602  0.94161013  1.37093366  0.12637073
  1.67484757  1.08621058]
SSE: 29.758415856417546 grad-norm: 0.008008011610891 parameters [-0.02224569  0.9842183   0.88351141  0.94163744  1.37095331  0.12637468
  1.67488046  1.08623564]
SSE: 29.75841534383787 grad-norm: 0.007994078114460002 parameters [-0.02224527  0.9842428   0.88353676  0.9416647   1.37097292  0.12637863
  1.67491329  1.08626067]
SSE: 29.75841483304036 grad-norm: 0.007980168861540914 parameters [-0.02224484  0.98426726  0.88356206  0.94169192  1.3709925

In [11]:
print(LA.inv(new_train.T @ new_train) @ new_train.T @ train_y)

[-0.02200118  0.99829962  0.89807839  0.95730469  1.38222513  0.12864071
  1.69374719  1.10061708]


In [51]:
from random import randint
# Setup for gradient descent
def new_gradient(data, model, actual_y):
  resid = 0
  resid += (model.T @ model - actual_y) * data
  return 2 * resid
param = np.zeros(new_train.shape[1])
queue = []
gamma = 0.03
tao = 0.0000001
num_iter = 5_000
for i in range(num_iter):
  index = randint(0, len(new_train)-1)
  SSE = total_cost_function(new_train, param, train_y)
  grad = new_gradient(np.array([new_train[index]]), param, train_y[index])
  grad_norm = LA.norm(grad)
  param = param - gamma * grad
  print("SSE:", SSE, "grad-norm:", grad_norm, "parameters", param)


  # queue.append(LA.norm(gradient(new_train[index], param, train_y[index])))
  # if len(queue) == 10 and np.mean(queue) < tao:
  #   break
  # if len(queue) == 10:
  #   queue.pop(0)

SSE: 44.65026827926844 grad-norm: 12.988085195578396 parameters [[-0.1243548  -0.16176321  0.1611576  -0.03070071  0.22983876  0.11304647
  -0.02265011 -0.1309854 ]]


ValueError: ignored